In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import seaborn as sns
from statsmodels.formula.api import ols

In [ ]:
air=pd.read_csv("/kaggle/input/air-quality-data-brisbane-cbd/brisbanecbdaq2017.csv")

In [ ]:
air.info()

# no null values are present
# all attributes are of float data type except Date and Time

In [ ]:
air.replace("inf","NaN",inplace=True)
air.dropna(axis=0,how="any",inplace=True)

# let us separate out first day month and year from date

In [ ]:
air["Date"]=pd.to_datetime(air.Date)
air=air.assign(day=air.Date.dt.day,month=air.Date.dt.month,year=air.Date.dt.year)
air.drop("Date",axis=1,inplace=True)

In [ ]:
air.info()

# day, month and year are int64 data type
# let us look at the unique values of day, month and year

In [ ]:
air["day"].value_counts()

# the data is less for the 31st day because few  months have 31 days

In [ ]:
air["month"].value_counts()

# data is minimum for the month 2 which is feb and hence less days the data is maximum for the months having 31 days

In [ ]:
air["year"].value_counts()

# we can see air quality data is only for the 2017 year and hence we can remove the year from the analysis

In [ ]:
air.drop("year",axis=1,inplace=True)
air["Time"].value_counts()

# we can air quality data is recorded everyday at an interval of one hour and let us extract the hour from the time

In [ ]:
air["Time"]=pd.to_datetime(air.Time)
air=air.assign(hour=air.Time.dt.hour)
air.drop("Time",axis=1,inplace=True)

# Let u findout the correlation between different parameters/attribute

In [ ]:
corr_matrix=air.corr()
print(corr_matrix["PM10 (ug/m^3)"])

# we can PM10 and visiblity reducing particles have strong +ve correlation. while PM10 has very small and negative correlation with wind direction and relative humidity
# All other parameters does not have any significant correlation with PM10
# We can also see this with the help of heat map

In [ ]:
sns.heatmap(corr_matrix)

In [ ]:
sns.lmplot(y="PM10 (ug/m^3)",x="Visibility-reducing Particles (Mm^-1)",data=air,fit_reg=False)

# let us find a model to predict the PM10 particles in the air

In [ ]:
air.rename(columns={'Wind Direction (degTN)':'WD','Wind Speed (m/s)':'WS','Wind Sigma Theta (deg)':'WST','Wind Speed Std Dev (m/s)':'WSS','Air Temperature (degC)':'AT','Relative Humidity (%)':'RH','Rainfall (mm)':'RF','Barometric Pressure (hPa)':'BP','PM10 (ug/m^3)':'PM10','Visibility-reducing Particles (Mm^-1)':'VRP'},inplace=True)
model=ols('PM10~WD+WS+VRP+RH+AT+VRP*RH*WD-1',data=air).fit()
model.summary()